# Pandas (2)

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

In [133]:
import pandas as pd
    
df = pd.read_csv('sp500hst.txt', sep=',', header=None, names=["date", "ticker", "open", "high", "low", "close", "volume"])
df

,date,ticker,open,high,low,close,volume
0,20090821,A,25.60,25.6100,25.220,25.55,34758
1,20090824,A,25.64,25.7400,25.330,25.50,22247
2,20090825,A,25.50,25.7000,25.225,25.34,30891
3,20090826,A,25.32,25.6425,25.145,25.48,33334
4,20090827,A,25.50,25.5700,25.230,25.54,70176
...,...,...,...,...,...,...,...
122569,20100813,ZMH,51.72,51.9000,51.380,51.44,14561
122570,20100816,ZMH,51.13,51.4700,50.600,51.00,13489
122571,20100817,ZMH,51.14,51.6000,50.890,51.21,20498
122572,20100819,ZMH,51.63,51.6300,50.170,50.22,18259


2. Посчитайте количество уникальных цифр, которые используются каждой строке в столбце volume.

In [134]:
def n_digits(volume):
    return len(set(str(volume)))

In [135]:
df["volume"].map(n_digits)

0         5
1         3
2         5
3         2
4         4
         ..
122569    4
122570    5
122571    5
122572    5
122573    4
Name: volume, Length: 122574, dtype: int64

3. Для каждой строки рассчитайте разность между значениями high и low, если индекс столбца нечетный, и разность между close и high в противном случае.

In [136]:
# Способ хуже

In [137]:
df.head()

,date,ticker,open,high,low,close,volume
0,20090821,A,25.60,25.6100,25.220,25.55,34758
1,20090824,A,25.64,25.7400,25.330,25.50,22247
2,20090825,A,25.50,25.7000,25.225,25.34,30891
3,20090826,A,25.32,25.6425,25.145,25.48,33334
4,20090827,A,25.50,25.5700,25.230,25.54,70176


In [138]:
df.loc[0].name

0

In [139]:
def diff(row):
    if row.name % 2:
        return row["high"] - row["low"]
    else:
        return row["close"] - row["high"]

In [140]:
df.apply(diff, axis=1)

0        -0.0600
1         0.4100
2        -0.3600
3         0.4975
4        -0.0300
           ...  
122569    0.5200
122570   -0.4700
122571    0.7100
122572   -1.4100
122573    1.0700
Length: 122574, dtype: float64

In [141]:
# Способ лучше

In [142]:
import numpy as np

np.where(df.index % 2 == 1, df["high"] - df["low"], df["close"] - df["high"])

array([-0.06,  0.41, -0.36, ...,  0.71, -1.41,  1.07])

4. Рассчитайте суммарный объем торгов для для одинаковых значений тикеров.

In [143]:
# df.groupby(np.unique(df["ticker"])).sum(df["volume"])
sum_df = df.groupby(["ticker"])["volume"].sum()
# sum_df["volume"]
sum_df

ticker
A        8609336
AA      81898998
AAPL    52261170
ABC      9006756
ABT     18975870
          ...   
XTO     21297931
YHOO    56837171
YUM     10971538
ZION    15551119
ZMH      4938916
Name: volume, Length: 524, dtype: int64

5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

## Лабораторная работа №2.2

In [17]:
import numpy as np
import pandas as pd

__Данная работа является продолжением ЛР №2. Для начала работы загрузите таблицы (см. задание 1.1)__

In [18]:
recipes = pd.read_csv("recipes_sample.csv")
reviews = pd.read_csv("reviews_sample.csv")

In [19]:
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [20]:
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [21]:
recipes.dtypes

name               object
id                  int64
minutes             int64
contributor_id      int64
submitted          object
n_steps           float64
description        object
n_ingredients     float64
dtype: object

### Применение функций к pd.Series и pd.DataFrame

4.1 Напишите функцию, которая переводит минуты в формат "XhYm". Примените эту функцию к столбцу `minutes` таблицы `recipes` (без перезаписи исходного столбца) при помощи метода `map`.

In [22]:
def m_to_hm(m: int) -> str:
    return f'{m//60}h{m%60}m'
    pass

assert m_to_hm(90) == "1h30m"
assert m_to_hm(10) == "0h10m"

In [23]:
recipes["minutes"].map(m_to_hm)

0        1h30m
1        0h10m
2        0h30m
3        0h45m
4        0h25m
         ...  
29995    1h20m
29996     4h0m
29997    1h15m
29998     1h0m
29999    0h29m
Name: minutes, Length: 30000, dtype: object

4.2 На основе таблицы `recipes` создайте таблицу, которая содержит только текстовые столбцы (используйте метод `select_dtypes`).  Примените к каждому элементу этой таблицы строковый метод `str.capitalize` при помощи метода `applymap`, не удаляя пропуски.

In [24]:
text_recipes = recipes.select_dtypes(include='object')
text_recipes = text_recipes[["name", "description"]]
text_recipes.applymap(lambda x: str(x).capitalize())

,name,description
0,George s at the cove black bean soup,An original recipe created by chef scott meska...
1,Healthy for them yogurt popsicles,My children and their friends ask for my homem...
2,I can t believe it s spinach,"These were so go, it surprised even me."
3,Italian gut busters,My sister-in-law made these for us at a family...
4,Love is in the air beef fondue sauces,I think a fondue is a very romantic casual din...
...,...,...
29995,Zurie s holey rustic olive and cheddar bread,This is based on a french recipe but i changed...
29996,Zwetschgenkuchen bavarian plum cake,"This is a traditional fresh plum cake, thought..."
29997,Zwiebelkuchen southwest german onion cake,This is a traditional late summer early fall s...
29998,Zydeco soup,This is a delicious soup that i originally fou...


4.3 Напишите функцию, которая принимает на вход серию `pd.Series` и для серий, содержащих текстовые данные, возвращает максимальную длину строк в ней, а для числовых серий возвращает минимальный элемент в этой серии. Примените данную функцию к каждому столбцу таблицы `recipes` при помощи метода `apply`.

In [25]:
def get_stats(x: pd.Series) -> int:
    if all(map(lambda x: isinstance(x, str), x)):
        return len(max(x, key=len))
    if all(map(lambda x: isinstance(x, int), x)):
        return min(x)


assert get_stats(pd.Series(['a', 'bbbb', 'ccc'])) == 4
assert get_stats(pd.Series([1, 3, 2])) == 1

In [26]:
result = recipes.apply(get_stats)
result

name                83.0
id                  48.0
minutes              0.0
contributor_id    1530.0
submitted           10.0
n_steps              NaN
description          NaN
n_ingredients        NaN
dtype: float64

### Группировки таблиц `pd.DataFrame`

5.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [27]:
amount_of_recipes = recipes.groupby("contributor_id")["id"].count().max()
df = recipes.groupby("contributor_id")["id"].count()
df[df == amount_of_recipes]

contributor_id
89831    421
Name: id, dtype: int64

5.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [28]:
rating_for_each_recipe = reviews.groupby("recipe_id")["rating"].mean()
rating_for_each_recipe

recipe_id
48        1.000000
55        4.750000
66        4.944444
91        4.750000
94        5.000000
            ...   
536547    5.000000
536610    0.000000
536728    4.000000
536729    4.750000
536747    0.000000
Name: rating, Length: 28100, dtype: float64

In [29]:
rating_for_each_recipe.isin([0]).sum()

660

5.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [30]:
reviews["date"] = pd.to_datetime(reviews['date'], format="%Y-%m-%d")
n_recipes_in_year = reviews.groupby(reviews["date"].dt.year)["recipe_id"].count()
n_recipes_in_year

date
2000       13
2001      305
2002     2494
2003     3879
2004     5070
2005     7577
2006     9964
2007    15634
2008    18614
2009    17672
2010    12094
2011     8118
2012     6095
2013     5679
2014     3445
2015     2715
2016     2071
2017     2926
2018     2331
Name: recipe_id, dtype: int64

5.4 Напишите функцию, которая принимает на вход таблицу (аналогичную `recipes` по набору столбцов), и возвращает `True` в том случае, если в столбце `minutes` присутствуют только значения, меньшие либо равные 10. Сгруппируйте таблицу `recipes` по полю `contributor_id` и для каждого участника выясните, справедливо ли, что все его рецепты занимают не более 10 минут.

In [31]:
def has_only_fast_recipes(x: pd.DataFrame) -> bool:
    if all(x['minutes'] <= 10):
        return True
    else:
        return False

assert not has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 90, 1: 10},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)
assert has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 7, 1: 5},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)

In [33]:
chk_fast_recipe = recipes.groupby("contributor_id").filter(has_only_fast_recipes)
chk_fast_recipe

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
44,emerald ecstasy green smoothie,342619,6,1010057,2008-12-09,9.0,this is the delicious green smoothie recipe th...,6.0
46,fraicheur vinaigrette,367987,5,797706,2009-04-26,NaN,this is a lovely salad dressing from a local g...,7.0
93,1 minute cake,290187,2,584365,2008-03-05,20.0,it's not a fancy cake but when you eat it warm...,NaN
109,10 minute chocolate self saucing pudding,402246,10,1464792,2009-12-02,9.0,"got this recipe from the ""essentially food mag...",11.0
...,...,...,...,...,...,...,...,...
29723,yum yum avocado salsa,296231,10,808017,2008-04-03,5.0,this is yummy! and healthy!,4.0
29724,yum yum balls,63973,5,88072,2003-06-06,3.0,these chocolate balls are delicious! my family...,NaN
29754,yummy honey and pear sandwiches,171091,2,318506,2006-06-02,3.0,i went looking for sandwich fillings for my on...,4.0
29789,yummy sour cream fruit dip,200350,5,57562,2006-12-12,3.0,mock jason's deli fruit dip. i've made it and ...,3.0


### Объединение таблиц `pd.DataFrame`

6.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [34]:
merged = recipes[["id","name"]].merge(reviews[["user_id", "recipe_id","rating"]], left_on='id', right_on='recipe_id')
merged = merged.drop(labels = "recipe_id", axis = 1)
merged

,id,name,user_id,rating
0,44123,george s at the cove black bean soup,743566,5
1,44123,george s at the cove black bean soup,76503,5
2,44123,george s at the cove black bean soup,34206,5
3,67664,healthy for them yogurt popsicles,494084,5
4,67664,healthy for them yogurt popsicles,303445,5
...,...,...,...,...
126691,486161,zydeco soup,305531,5
126692,486161,zydeco soup,1271506,5
126693,486161,zydeco soup,724631,5
126694,486161,zydeco soup,133174,5


In [35]:
reviews[reviews['review'].isna()].index

Int64Index([  8881,  12129,  18578,  21913,  33232,  42783,  42855,  43551,
             53998,  55100,  56197,  59803,  67973,  71067,  96199, 105206,
            109590],
           dtype='int64')

In [36]:
reviews.loc[[12129]]

,Unnamed: 0,user_id,recipe_id,date,rating,review
12129,273481,2002120930,167202,2018-04-18,4,NaN


In [37]:
merged.loc[[12129]]

,id,name,user_id,rating
12129,304615,belgian carrots,53932,5


6.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [38]:
reviews["review_count"] = reviews.groupby("recipe_id")["rating"].count()
new_merged = reviews[["recipe_id", "review_count"]].merge(recipes[["id","name"]],left_on='recipe_id', right_on='id')
new_merged = new_merged.drop(labels = "id", axis = 1)
new_merged["review_count"].fillna(0, inplace=True)
new_merged

,recipe_id,review_count,name
0,57993,0.0,salmon with tomatoes
1,57993,0.0,salmon with tomatoes
2,57993,0.0,salmon with tomatoes
3,57993,0.0,salmon with tomatoes
4,57993,0.0,salmon with tomatoes
...,...,...,...
126691,94096,0.0,vegetable pasta pie
126692,368208,0.0,crockery pot delight
126693,154964,0.0,apple cider cheesecake no cook
126694,187418,0.0,winter vegetable lasagna


6.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [39]:
least_mean_rating = reviews["rating"].groupby(reviews["date"].dt.year).mean().min()
df = reviews["rating"].groupby(reviews["date"].dt.year).mean()
df[df == least_mean_rating]

date
2017    3.353042
Name: rating, dtype: float64

### Сохранение таблиц `pd.DataFrame`

7.1 Отсортируйте результат выполнения задания 6.1 в порядке убывания величины столбца `id` и сохраните результаты в csv файл. 

In [40]:
to_write = merged.sort_values(by=["id"])
to_write.to_csv('merged_file.csv')

7.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 6.1 и 6.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 6.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 6.2.

In [41]:
with pd.ExcelWriter("result_of_work.xlsx") as writer:
    merged.to_excel(writer, sheet_name="Рецепты с оценками")  
    new_merged.to_excel(writer, sheet_name="Количество отзывов по рецептам")